In [103]:
import requests
import time
import pymysql
import os
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [56]:
def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp

### API approach, has limits per day - not worth it, continue to full scraping approach

In [2]:
'''
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
'''
    

Requestiong:  poloclub cnn-explainer
https://api.github.com/repos/poloclub/cnn-explainer/contributors
user "xiaohk" has 189 contributions
https://github-user.now.sh/?username=xiaohk


TypeError: 'Response' object is not subscriptable

### Saving author contributions to txt file

In [114]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

In [115]:

repos = []
cursor.execute("SELECT DISTINCT krw.repo.repo_url FROM krw.paper \
INNER JOIN krw.repo ON krw.repo.paper_url = krw.paper.paper_url \
WHERE krw.paper.makg_sameAs <> 'notfound'")
for r in cursor:
    repos.append(list(r)[0])
'''
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
'''

'\ncursor.execute("SELECT krw.repo.paper_url, repo_url, author                FROM krw.repo INNER JOIN krw.paper_author                ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")\nfor r in cursor:\n    data.append(list(r))\n'

In [116]:
len(repos)

11266

In [120]:
repos[0:10]

['https://github.com/twke18/Adaptive_Affinity_Fields',
 'https://github.com/VladyslavUsenko/basalt-mirror',
 'https://github.com/new-okaerinasai/bwgan_pytorch',
 'https://github.com/ManuelVs/NeuralNetworks',
 'https://github.com/ruotianluo/DiscCaptioning',
 'https://github.com/nitika-verma/FeaStNet',
 'https://github.com/tfolkman/deep-learning-experiments',
 'https://github.com/csliangdu/FSASL',
 'https://github.com/loshchil/SGDR',
 'https://github.com/minhhuu291/Logo-Recognition']

In [121]:
# Remove repose we already saved into file
with open(filename) as fp:
    line = fp.readline().split()
    while line:
        current_repo = line[0]
        if current_repo in repos:
            repos.remove(current_repo)
        line = fp.readline().split()
len(repos)

11052

In [119]:
repos.index('https://github.com/facebookresearch/InferSent')

150

In [69]:
filename = 'scraped_data.txt'
if not os.path.exists(filename):
    file = open(filename, 'w+')
    file.close()

In [129]:
missed = []

['https://github.com/YanWei123/Pointnet_encoder_Foldingnet_decoder_quantization']

In [130]:
driver = webdriver.Firefox() #Chrome(ChromeDriverManager().install())

for repo in repos[500:1000]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/tfrerix/proxprop
https://github.com/tfrerix
name: Thomas Frerix, bio: 
User tfrerix: commits 3 , changed lines + 1,185 - 80
https://github.com/moellenh
name: Thomas Möllenhoff, bio: 
User moellenh: commits 1 , changed lines + 1 - 1

https://github.com/cantaible/tf-faster-rcnn-note
https://github.com/cantaible
name: , bio: 
User cantaible: commits 4 , changed lines + 20,098 - 26

https://github.com/talal579/Deep-image-matching
https://github.com/talalsoms
name: Talal Ahmed, bio: 
User talalsoms: commits 6 , changed lines + 1,565 - 5

https://github.com/lmzintgraf/gp_pref_elicit
https://github.com/lmzintgraf
name: Luisa M Zintgraf, bio: 
User lmzintgraf: commits 8 , changed lines + 6,725 - 24

https://github.com/Stick-To/Inception-tensorflow
https://github.com/Stick-To
name: , bio: 
User Stick-To: commits 17 , changed lines + 1,648 - 427

https://github.com/Franck-Dernoncourt/naacl2016
https://github.com/Franck-Dernoncourt
name: Franck Dernoncourt, bio: 
User Franck-De

https://github.com/Non-biri
name: Non, bio: Tokyo University of Technology student
User Non-biri: commits 139 , changed lines + 185,560 - 1,672

https://github.com/anhinga/typed-dmms
https://github.com/anhinga
name: , bio: 
User anhinga: commits 9 , changed lines + 121 - 36

https://github.com/palmagro/gg2vec
https://github.com/palmagro
name: , bio: 
User palmagro: commits 105 , changed lines + 21,255 - 19,247

https://github.com/nd7141/AWE
https://github.com/nd7141
name: Sergey Ivanov, bio: 
User nd7141: commits 76 , changed lines + 4,256 - 27,448
https://github.com/sergeyivanovcriteo
name: Sergey Ivanov, bio: 
User sergeyivanovcriteo: commits 20 , changed lines + 49,681 - 373

https://github.com/jsheng7/ANI1-qm7
https://github.com/jsheng7
name: , bio: 
User jsheng7: commits 9 , changed lines + 2,649,918 - 104

https://github.com/jnhwkim/ban-vqa
https://github.com/jnhwkim
name: Jin-Hwa Kim, bio: Research Scientist
User jnhwkim: commits 25 , changed lines + 2,863 - 134
https://github.c

name: Simon Hakenes, bio: 
User shakenes: commits 2 , changed lines + 2 - 1
https://github.com/olegklimov
name: Oleg Klimov, bio: Engineer at OpenAI
User olegklimov: commits 2 , changed lines + 23 - 22
https://github.com/louiehelm
name: Louie Helm, bio: Machine Learning Researcher
User louiehelm: commits 2 , changed lines + 4 - 4
https://github.com/christopherhesse
name: Christopher Hesse, bio: 
User christopherhesse: commits 2 , changed lines + 89 - 717
https://github.com/20chase
name: , bio: 
User 20chase: commits 2 , changed lines + 5 - 9
https://github.com/zach-nervana
name: Zach Dwiel, bio: 
User zach-nervana: commits 1 , changed lines + 0 - 1
https://github.com/yenchenlin
name: Yen-Chen Lin, bio: PhD student at MIT CSAIL
User yenchenlin: commits 1 , changed lines + 3 - 3
https://github.com/welinder
name: Peter Welinder, bio: 
User welinder: commits 1 , changed lines + 2 - 2

https://github.com/marfoq/unsupervised_translator
NOT FOUND/LOADED

https://github.com/x5675602/LSUVinit-k

name: Christopher Hesse, bio: 
User christopherhesse: commits 1 , changed lines + 2 - 0

https://github.com/weiyuk/fluent-cap
NOT FOUND/LOADED

https://github.com/InnovArul/personreid_normxcorr
NOT FOUND/LOADED

https://github.com/AlexKuhnle/film
NOT FOUND/LOADED

https://github.com/mhauskn/dqn-hfo
NOT FOUND/LOADED

https://github.com/wenguanwang/ASNet
NOT FOUND/LOADED

https://github.com/Lzsxx/Leaky-Reluplex
NOT FOUND/LOADED

https://github.com/yw0nam/DenseNet
NOT FOUND/LOADED

https://github.com/JeffreyJackovich/bitcoin-prediction-with-sentiment
NOT FOUND/LOADED

https://github.com/Enderdead/Pytorch_Quantize_impls
NOT FOUND/LOADED

https://github.com/d909b/DSMT-Nets
NOT FOUND/LOADED

https://github.com/yinanzhu12/SegNet-keras-implementation
NOT FOUND/LOADED

https://github.com/kamperh/recipe_bucktsong_awe_py3
NOT FOUND/LOADED

https://github.com/rnirdhar/yoloMultiClass
NOT FOUND/LOADED

https://github.com/zihangdai/mos
https://github.com/zihangdai
name: Zihang Dai, bio: PhD student i

NOT FOUND/LOADED

https://github.com/danielbis/python_cheap_fashion
https://github.com/danielbis
name: Daniel Bis, bio: PhD. student at Florida State University
User danielbis: commits 11 , changed lines + 48,337 - 123
https://github.com/sm415
name: Sam, bio: 
User sm415: commits 4 , changed lines + 222 - 26
https://github.com/Rehluct
name: Steven Centeno, bio: FSU Student 

#gonoles
User Rehluct: commits 1 , changed lines + 301 - 0

https://github.com/mahmoudnabil/ASTD
NOT FOUND/LOADED

https://github.com/Ravi-Garg/Unsupervised_Depth_Estimation
NOT FOUND/LOADED

https://github.com/ferhatkkochan/deeplearning-notes
NOT FOUND/LOADED

https://github.com/radoslawkrolikowski/sentiment-analysis-pytorch
NOT FOUND/LOADED

https://github.com/kaishengtai/neuralart
https://github.com/kaishengtai
name: Kai Sheng Tai, bio: Ph.D. student at Stanford
User kaishengtai: commits 22 , changed lines + 1,184 - 444
https://github.com/bobbens
name: Edgar Simo-Serra, bio: 
User bobbens: commits 5 , changed li

...

name: Xiang Ren, bio: Asst. Professor, University of Southern California
User shanzhenren: commits 7 , changed lines + 73 - 32
https://github.com/ellenmellon
name: Zeqiu (Ellen) Wu, bio: PhD Student at UW NLP Research Group
User ellenmellon: commits 3 , changed lines + 74 - 32

https://github.com/sleepinyourhat/vector-entailment
NOT FOUND/LOADED

https://github.com/Hanlard/Bert-for-WebQA
NOT FOUND/LOADED

https://github.com/jasonlaska/spherecluster
https://github.com/jasonlaska
name: Jason Laska, bio: Topics: Compressive Sensing, Computer Vision, and Natural Language. Helper at @clara-labs | @skggm  | @rejecta | @dropcam
User jasonlaska: commits 24 , changed lines + 972 - 763
https://github.com/dominikstrb
name: Dominik Straub, bio: PhD student at @RothkopfLab
User dominikstrb: commits 1 , changed lines + 12 - 12
https://github.com/cv3d
name: Hamdi Sahloul, bio: 
User cv3d: commits 1 , changed lines + 32 - 8

https://github.com/FloList/21cmGAN
NOT FOUND/LOADED

https://github.com/yaoli

NOT FOUND/LOADED

https://github.com/geopavlakos/ordinal-pose3d
NOT FOUND/LOADED

https://github.com/MNaplesDevelopment/DCGAN-PyTorch
NOT FOUND/LOADED

https://github.com/ibalpowr/hello-srrc2016
NOT FOUND/LOADED

https://github.com/raminmh/LSTM_dynamics_interpretability
NOT FOUND/LOADED

https://github.com/sz128/NLU_datasets_for_task_oriented_dialogue
NOT FOUND/LOADED

https://github.com/msmsajjadi/frvsr
https://github.com/msmsajjadi
name: Mehdi S. M. Sajjadi, bio: Google Research
User msmsajjadi: commits 19 , changed lines + 100 - 23
https://github.com/rzumer
name: Raphaël Zumer, bio: B. Eng.
User rzumer: commits 1 , changed lines + 1 - 1

https://github.com/kaixiao/PLNN
NOT FOUND/LOADED

https://github.com/IBM/ZOSVRG-BlackBox-Adv
NOT FOUND/LOADED

https://github.com/statisticalplumber/Building_detection
NOT FOUND/LOADED

https://github.com/singhsidhukuldeep/Generative-Adversarial-Network-GAN
https://github.com/singhsidhukuldeep
name: Kuldeep Singh Sidhu, bio: Always exploring 😛
User 

name: , bio: A teen who is passionate about coding and the open source movement.
I am fluent(ish) in python.

https://medium.com/@geek_kid
User qwertpi: commits 1 , changed lines + 5 - 5
https://github.com/nsarang
name: Nima Sarang, bio: Data Scientist, Researcher
User nsarang: commits 1 , changed lines + 1 - 1
https://github.com/girishponkiya
name: Girishkumar, bio: PhD in NLP from IIT Bombay, Mumbai.
User girishponkiya: commits 1 , changed lines + 6 - 1
https://github.com/amlarraz
name: , bio: 
User amlarraz: commits 1 , changed lines + 1 - 1
https://github.com/acere
name: Alessandro Cere, bio: Data, image processing, mostly Python, but also C, Matlab, and more.
User acere: commits 1 , changed lines + 36 - 0
https://github.com/JonnoFTW
name: Jonathan Mackenzie, bio: PhD Student in focusing on datamining and machine learning for traffic analysis. I like to make cool things in my spare time.
User JonnoFTW: commits 1 , changed lines + 17 - 8

https://github.com/kevinduh/san_mrc
https://

name: JP, bio: 
User jpchen: commits 225 , changed lines + 21,907 - 10,200
https://github.com/martinjankowiak
name: , bio: 
User martinjankowiak: commits 221 , changed lines + 25,693 - 12,179
https://github.com/fehiepsi
name: Du Phan, bio: a math student, a data novice, a zen mind
User fehiepsi: commits 141 , changed lines + 19,003 - 7,584
https://github.com/stefanwebb
name: Stefan Webb, bio: Applied Machine Learning Researcher
User stefanwebb: commits 40 , changed lines + 8,794 - 4,418
https://github.com/null-a
name: , bio: 
User null-a: commits 21 , changed lines + 2,661 - 557
https://github.com/karalets
name: , bio: 
User karalets: commits 18 , changed lines + 1,713 - 354
https://github.com/ae-foster
name: , bio: Oxford DPhil student in Statistical Machine Learning
User ae-foster: commits 14 , changed lines + 7,278 - 2,688
https://github.com/rohitsingh0812
name: Rohit Singh, bio: 
User rohitsingh0812: commits 10 , changed lines + 316 - 248
https://github.com/ahmadsalim
name: Ahmad S

KeyboardInterrupt: 

In [126]:
len(missed)

44

### Getting list of repositories and authors from the file

In [77]:
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))

In [79]:
author_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

1148


In [86]:
author_dict['https://github.com/google/tf_mesh_renderer']

['Kyle Genova',
 'Forrester Cole',
 'Aaron Maschinot',
 'Aaron Sarna',
 'Daniel Vlasic',
 'William T. Freeman']

In [84]:
filename_mini = 'scraped_data_mini.txt' # use full version later

In [89]:
paper_data = {}

with open(filename_mini) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[current_repo] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[current_repo][author_name] = cont[2:]
            # after saving previous authors, load new repo
            current_repo = line[0]
            contributors = [line[1:]]                    

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

IndexError: list index out of range

In [90]:
paper_data

{'https://github.com/google/tf_mesh_renderer': {'Kyle Genova': ['Kyle_Genova',
   '2',
   '18',
   '1']},
 'https://github.com/twke18/Adaptive_Affinity_Fields': {'Tsung-Wei Ke': ['Tsung-Wei_Ke',
   '2',
   '24',
   '6'],
  'Jyh-Jing Hwang': ['Jyh-Jing_Hwang', '9', '31', '26']},
 'https://github.com/VladyslavUsenko/basalt-mirror': {'Vladyslav Usenko': ['Vladyslav_Usenko',
   '227',
   '66,951',
   '10,162'],
  'Nikolaus Demmel': ['Nikolaus_Demmel', '24', '662', '1,219']},
 'https://github.com/new-okaerinasai/bwgan_pytorch': {},
 'https://github.com/ManuelVs/NeuralNetworks': {},
 'https://github.com/ruotianluo/DiscCaptioning': {},
 'https://github.com/nitika-verma/FeaStNet': {'Nitika Verma': ['Nitika_Verma',
   '5',
   '344',
   '11']},
 'https://github.com/tfolkman/deep-learning-experiments': {},
 'https://github.com/csliangdu/FSASL': {},
 'https://github.com/loshchil/SGDR': {'Ilya Loshchilov': ['Ilya_Loshchilov',
   '4',
   '461',
   '2']},
 'https://github.com/minhhuu291/Logo-Recognit

### Full scraping approach

In [61]:
paper_data = {}
driver = webdriver.Firefox()

for paper in final_data:
    repo = paper[1]
    print(repo, paper[2]) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:10]:   # take just top 10 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name} and {type(name)}, bio: {bio}')
        
        if name!="" and name in paper[2]:
            if paper[0] not in paper_data:
                paper_data[paper[0]]= [paper[1], {}]
            #print(info_line[1])
            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            paper_data[paper[0]][1][name] = [commits, added, deleted]
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/nianticlabs/monodepth2 ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']
https://github.com/mdfirman
name:  and <class 'str'>, bio: 
https://github.com/mrharicot
name: Clément Godard and <class 'str'>, bio: 
https://github.com/daniyar-niantic
name: Daniyar Turmukhambetov and <class 'str'>, bio: Senior Research Scientist at Niantic

https://github.com/google/tf_mesh_renderer ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
https://github.com/fcole
name:  and <class 'str'>, bio: 
https://github.com/kylegenova
name: Kyle Genova and <class 'str'>, bio: 
User kylegenova: commits 2 , changed lines + 18 - 1
https://github.com/TerisGH
name:  and <class 'str'>, bio: 

https://github.com/Panda-Peter/visda2019-multisource ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']
https://github.com/Panda-Peter
name:  and <class 'str'>, bio: Yingwei Pan

https://github.com/yanggeng1995/GAN-TTS ['Mikołaj Bińk


https://github.com/DylanWusee/pointconv_pytorch ['Charles R. Qi', 'Hao Su', 'Kaichun Mo', 'Leonidas J. Guibas']


WebDriverException: Message: Failed to decode response from marionette


In [47]:
paper_data

{'https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model': ['https://github.com/google/tf_mesh_renderer',
  {'Kyle Genova': ['2', '18', '1']}],
 'https://paperswithcode.com/paper/adaptive-affinity-fields-for-semantic': ['https://github.com/twke18/Adaptive_Affinity_Fields',
  {'Tsung-Wei Ke': ['2', '24', '6'], 'Jyh-Jing Hwang': ['9', '31', '26']}],
 'https://paperswithcode.com/paper/improving-information-retrieval-results-for': ['https://github.com/adelra/query-expansion',
  {'Adel Rahimi': ['2', '99,652', '0']}],
 'https://paperswithcode.com/paper/the-double-sphere-camera-model': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'https://paperswithcode.com/paper/the-tum-vi-benchmark-for-evaluating-visual': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'ht

In [3]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [4]:
driver = webdriver.Firefox()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        print()

https://github.com/poloclub/cnn-explainer
User xiaohk: commits 189 , changed lines + 35,562 - 14,943
https://github.com/xiaohk
name: Jay Wang, bio: Machine Learning PhD student at Georgia Tech.


Life is not always 🦄 and 🌈, but my favorite emoji is 😉

User rturko3: commits 58 , changed lines + 2,870 - 950
https://github.com/rturko3
name: , bio: CS Undergraduate @ Georgia Tech

User oshaikh13: commits 8 , changed lines + 117 - 74
https://github.com/oshaikh13
name: Omar Shaikh, bio: no u

https://github.com/openai/jukebox
User prafullasd: commits 49 , changed lines + 54,045 - 301
https://github.com/prafullasd
name: Prafulla Dhariwal, bio: 

User heewooj: commits 24 , changed lines + 393 - 344
https://github.com/heewooj
name: Heewoo Jun, bio: 

User jongwook: commits 5 , changed lines + 18 - 13
https://github.com/jongwook
name: Jong Wook Kim, bio: 

User mcleavey: commits 4 , changed lines + 964 - 3
https://github.com/mcleavey
name: Christine Payne, bio: OpenAI Researcher. Stanford Master